In [ ]:
import os


In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [ ]:
from src.textSummerizer.constants import *
from src.textSummerizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,config_path=CONFIG_FILE_PATH,
             params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params =read_yaml(params_filepath)
        
        create_directories([self.config.artifact_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            
                root_dir= config.root_dir,
                data_path=config.data_path,
                model_ckpt= config.model_ckpt,
                num_train_epochs= params.num_train_epochs,
                warmup_steps= params.warmup_steps,
                per_device_train_batch_size= params.per_device_train_batch_size ,
                weight_decay= params.weight_decay ,
                logging_steps= params.logging_steps ,
                evaluation_strategy= params.evaluation_strategy ,
                eval_steps= params.eval_steps ,
                save_steps= params.save_steps ,
                gradient_accumulation_steps=params.gradient_accumulation_steps
        )
        
        return model_trainer_config

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_from_disk

In [ ]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config= config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus =AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading the data
        datasest_samsum_pt =load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=10,             # Number of times to iterate over the entire dataset
            warmup_steps=500,               # Steps for learning rate warmup before training starts
            per_device_train_batch_size=1,  # Batch size during training (per device/GPU)
            per_device_eval_batch_size=1,   # Batch size during evaluation (per device/GPU)
            weight_decay=0.01,              # L2 regularization to prevent overfitting
            logging_steps=10,               # Log metrics every 10 steps
            evaluation_strategy='steps',    # Evaluation occurs at specific steps (not after each epoch)
            eval_steps=500,                 # Run evaluation every 500 steps
            save_steps=1e6,                 # Save model checkpoint after 1 million steps (effectively never during training)
            gradient_accumulation_steps=16  # Accumulate gradients over 16 steps before updating weights
        )
        dataset_samsum_pt =load_from_disk(self.config.data_path)
        
        trainer = Trainer(model = model_pegasus,
                  args = trainer_args,
                  tokenizer =tokenizer,data_collator=seq2seq_data_collator,
                  train_dataset =dataset_samsum_pt['test'],
                  eval_dataset = dataset_samsum_pt['validation'])

        trainer.train()
        
        #Saving the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        
        #saving the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

        

In [ ]:
!pip install --upgrade accelerate
!pip unistall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
config =ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config =model_trainer_config)
model_trainer.train()